For static replication of any constant maturity swap (CMS) payoff $g(F)$, where $F$ is the swap rate, we use the following formula:

$$\begin{aligned}
      V_0 &= D(0,T) g(F) + h'(F)[V^{pay}(F)-V^{rec}(F)] \\
      &\;\;\;\;\;\;\;\;\;\;+ \int_0^F h''(K) V^{rec}(K) dK +
      \int_F^\infty h''(K) V^{pay}(K) dK
\end{aligned}$$

where

  \begin{equation*}
    \begin{split}
      h(K) &= \frac{g(K)}{\text{IRR}(K)} \\
      h'(K) &= \frac{\text{IRR}(K)g'(K) - g(K)\text{IRR}'(K)}{\text{IRR}(K)^2} \\
      h''(K) &= \frac{\text{IRR}(K)g''(K)-\text{IRR}''(K)g(K) -2\cdot\text{IRR}'(K)g'(K)}{\text{IRR}(K)^2} \\
      &\;\;\;\;\;\;\;\;\;\;+
      \frac{2\cdot\text{IRR}'(K)^2g(K)}{\text{IRR}(K)^3}.
    \end{split}
  \end{equation*}
  
For CMS rate payoff, the payoff function can be defined simply as $g(F)=F$, and the static replication formula simplifies into:

  \begin{equation*}
    \begin{split}
      D(0,T) F + \int_0^F h''(K) V^{rec}(K) dK + \int_F^\infty h''(K) V^{pay}(K) dK
    \end{split}
  \end{equation*}

Let $m$ denote the payment frequency ($m=2$ for semi-annual payment frequency), and let $N = T_N-T_n$ denote the tenor of the swap (number of years), the partial derivatives on the IRR function $\text{IRR}(S)$ given by:
\begin{equation*}
\begin{split}
\text{IRR}(K)&=\sum_{i=1}^{N\times m}\frac{1}{(1+\frac{K}{m})^i}=\frac{1}{K}\left[1-\frac{1}{\left(1+\frac{K}{m}\right)^{N\times m}}\right]\\
\text{IRR}'(K)&=-\frac{1}{K}\text{IRR}(K)
+\frac{1}{m\times K}\frac{N\times m}{\left(1+\frac{K}{m}\right)^{N\times m+1}} \\
\text{IRR}''(K)&=-\frac{2}{K}\text{IRR}'(K)
-\frac{1}{m^2\times K}\frac{N\times m\cdot (N\times m+1)}{\left(1+\frac{K}{m}\right)^{N\times m+2}} \\
\end{split}
\end{equation*}

These results will need to be generalised to handle the case for $m=2$ to be consistent with the semi-annual payment frequency swap market data provided.

---
For CMS rate payment, since $g(F)=F$, we have the derivatives:

\begin{equation*}
\begin{split}
g(K) &= K \\
g'(K) &= 1 \\
g''(K) &= 0
\end{split}
\end{equation*}

# Imports and data wrangling

OIS is in semiannual df. need to also include for quarterly period

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy.optimize import brentq
from scipy.integrate import quad
from scipy.stats import norm
from scipy.interpolate import interp1d

ois = pd.read_csv('./data/ois.csv')     # load discounting curve
fsr = pd.read_csv('./data/sabr_params.csv')     # load discounting curve
# get discount factors up to quarterly periods
new_expiry = np.arange(0.25, ois.years.max()+.25, .25)
ois = ois.set_index("years").reindex(new_expiry)
ois.f = ois.f.bfill()
ois.reset_index(inplace=True)
ois["df"] = 1 / (1 + ois["f"] / 360) ** (90)  # get quarterly DF
ois["df"] = ois["df"].cumprod()               # cum prod to get all DF

In [2]:
## IRR funcions
def IRR_0(K, m, N):
    # implementation of IRR(K) function
    value = 1/K * ( 1.0 - 1/(1 + K/m)**(N*m) )
    return value

def IRR_1(K, m, N):
    # implementation of IRR'(K) function (1st derivative)
    firstDerivative = -1/K*IRR_0(K, m, N) + 1/(K*m)*N*m/(1+K/m)**(N*m+1)
    return firstDerivative

def IRR_2(K, m, N):
    # implementation of IRR''(K) function (2nd derivative)
    secondDerivative = -2/K*IRR_1(K, m, N) - 1/(K*m*m)*(N*m)*(N*m+1)/(1+K/m)**(N*m+2)
    return secondDerivative
# Payoff functions
def g_0(K):
    return K

def g_1(K):
    return 1.0

def g_2(K):
    return 0.0

def h_0(K, m, N):
    # implementation of h(K)
    value = g_0(K) / IRR_0(K, m, N)
    return value

def h_1(K, m, N):
    # implementation of h'(K) (1st derivative)
    firstDerivative = (IRR_0(K, m, N)*g_1(K) - g_0(K)*IRR_1(K, m, N)) / IRR_0(K, m, N)**2
    return firstDerivative

def h_2(K, m, N):
    # implementation of h''(K) (2nd derivative)
    secondDerivative = ((IRR_0(K, m, N)*g_2(K) - IRR_2(K, m, N)*g_0(K) - 2.0*IRR_1(K, m, N)*g_1(K))/IRR_0(K, m, N)**2 
                        + 2.0*IRR_1(K, m, N)**2*g_0(K)/IRR_0(K, m, N)**3)
    return secondDerivative
    
def black76_pay(F, K, expiry, tenor, sigma):
    """
    Return value of payer swaption via Black Normal model only
    parameters
        F: forward par swap rate given by forward IRS
        K: strike of swaption
        expiry: time to swaption expiry in years
        tenor: tenor of underlying swap in years
        sigma: annual vol
    """
    t = expiry
    d_1 = (np.log(F/K) + (1/2) * (sigma**2) * t) / (sigma * np.sqrt(t))
    d_2 = d_1 - sigma * np.sqrt(t)
    black_option = F * norm.cdf(d_1) - K * norm.cdf(d_2)
    return black_option
    
def black76_rec(F, K, expiry, tenor, sigma):
    """
    Return value of receiver swaption via Black Normal model Only
    parameters
        F: Forward par swap rate given by forward IRS
        K: strike of swaption
        expiry: time to swaption expiry in years
        tenor: tenor of underlying swap in years
        sigma: annual vol
    """
    t = expiry
    d_1 = (np.log(F/K) + (1/2) * (sigma**2) * t) / (sigma * np.sqrt(t))
    d_2 = d_1 - sigma * np.sqrt(t)
    black_option = K * norm.cdf(-d_2) - F * norm.cdf(-d_1)
    return black_option

def SABR(F, K, T, alpha, rho, nu, beta=0.9):
    """
    Use SABR model to return sigma, set beta to be 0.9
    """
    X = K
    # if K is at-the-money-forward
    if abs(F - K) < 1e-12:
        numer1 = (((1 - beta)**2)/24)*alpha*alpha/(F**(2 - 2*beta))
        numer2 = 0.25*rho*beta*nu*alpha/(F**(1 - beta))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        VolAtm = alpha*(1 + (numer1 + numer2 + numer3)*T)/(F**(1-beta))
        sabrsigma = VolAtm
    else:
        z = (nu/alpha)*((F*X)**(0.5*(1-beta)))*np.log(F/X)
        zhi = np.log((((1 - 2*rho*z + z*z)**0.5) + z - rho)/(1 - rho))
        numer1 = (((1 - beta)**2)/24)*((alpha*alpha)/((F*X)**(1 - beta)))
        numer2 = 0.25*rho*beta*nu*alpha/((F*X)**((1 - beta)/2))
        numer3 = ((2 - 3*rho*rho)/24)*nu*nu
        numer = alpha*(1 + (numer1 + numer2 + numer3)*T)*z
        denom1 = ((1 - beta)**2/24)*(np.log(F/X))**2
        denom2 = (((1 - beta)**4)/1920)*((np.log(F/X))**4)
        denom = ((F*X)**((1 - beta)/2))*(1 + denom1 + denom2)*zhi
        sabrsigma = numer/denom

    return sabrsigma

In [3]:
def rec_integrand(F, K, expiry, tenor, alpha, rho, nu, m):
    """
    Return value of receiver integrand per CMS rate payoff. 
    formula: 2nd derivative h * V_rec(per SABR valuation) 
    parameters:
        m: payment freq per year, e.g. 2 is semi-annual
        F: par swap rate
        K: strike of CMS
        expiry: years to expiry
        alpha: SABR param
        rho: SABR param
        nu: SABR param
    """
    sigma_sabr = SABR(F, K, expiry, alpha, rho, nu, beta=0.9)
    sabr_pricing = black76_rec(F, K, expiry, tenor, sigma_sabr)
    return h_2(K, m, tenor) * sabr_pricing

def pay_integrand(F, K, expiry, tenor, alpha, rho, nu, m):
    """
    Return value of receiver integrand per CMS rate payoff. 
    formula: 2nd derivative h * V_rec(per SABR valuation) 
    parameters:
        m: payment freq per year, e.g. 2 is semi-annual
        F: par swap rate
        K: strike of CMS
        expiry: years to expiry
        alpha: SABR param
        rho: SABR param
        nu: SABR param
    """
    sigma_sabr = SABR(F, K, expiry, alpha, rho, nu, beta=0.9)
    sabr_pricing = black76_pay(F, K, expiry, tenor, sigma_sabr)
    return h_2(K, m, tenor) * sabr_pricing

def v_0(F, expiry, tenor, alpha, rho, nu, m):
    """
    Return CMS pay off based of static replication discounted to today's price. Uses SABR model
    
    """
    df_expiry = ois.loc[ois.years == expiry]['df'].values[0]
    int_1 = IRR_0(F, m, tenor) * quad(lambda x: rec_integrand(F, x, expiry, tenor, alpha, rho, nu, m), 0.0, F)[0]
    int_2 = IRR_0(F, m, tenor) * quad(lambda x: pay_integrand(F, x, expiry, tenor, alpha, rho, nu, m), F, 0.1)[0]

    return df_expiry * F + int_1 + int_2



### Data prep for CMS 10y

In [ ]:


tenor = 10
tenor10_forward = fsr.loc[fsr.Tenor == tenor]
tenor10_forward = tenor10_forward.copy()  # Make an explicit copy

new_expiry = np.arange(0, 5.5, 0.5)

df_10y_cms = pd.DataFrame({"Expiry": new_expiry, 
                                        "Tenor" : 10})

# interpolate sabr params
for col in ["alpha", "rho", "nu"]:
    interp_func = interp1d(tenor10_forward["Expiry"], 
                        tenor10_forward[col], 
                        kind="linear", 
                        fill_value="extrapolate")
    df_10y_cms[col] = interp_func(new_expiry)
# add par IRS  then interpolate par swap rates
tenor10_forward.loc[0] = {"Expiry": 0, "f_irs": 0.037} # from excel sheet
tenor10_forward.sort_values(by="Expiry", inplace=True)
interp_func = interp1d(tenor10_forward["Expiry"], 
                    tenor10_forward['f_irs'], 
                    kind="linear", 
                    fill_value="extrapolate")
df_10y_cms['f_irs'] = interp_func(new_expiry)

df_10y_cms.head(2)

,Expiry,Tenor,alpha,rho,nu,f_irs
0,0.0,10,0.169848,0.225443,-0.847408,0.037000
1,0.5,10,0.170400,0.244767,-0.813154,0.037703


### Calculate PV of CMS 10y for 5 years

In [8]:
ls_v0 = []
for expiry in np.arange(0.0, 5, 0.5):
    if expiry == 0:
        coupon = df_10y_cms.iloc[0]['f_irs']
        ls_v0.append(coupon)       # first payment is fixed at par swap for first period
        continue

    F = df_10y_cms.loc[df_10y_cms.Expiry == expiry].f_irs.values[0]
    alpha = df_10y_cms.loc[df_10y_cms.Expiry == expiry].alpha.values[0]
    rho = df_10y_cms.loc[df_10y_cms.Expiry == expiry].rho.values[0]
    nu = df_10y_cms.loc[df_10y_cms.Expiry == expiry].nu.values[0]
    m = 2 # semi-annual

    coupon = v_0(F, expiry, tenor, alpha, rho, nu, m)
    print(f"{expiry}: {coupon}")
    ls_v0.append(coupon)
f"PV of a leg receiving CMS10y semi-annually over the next 5 years {np.array(ls_v0).sum():.5f}"

0.5: 0.03771245721216735
1.0: 0.03842456479512395
1.5: 0.03909078407628777
2.0: 0.03976026621756963
2.5: 0.04042549253563059
3.0: 0.041083811707227864
3.5: 0.04172531287879377
4.0: 0.04235758558930976
4.5: 0.042979920905893886


'PV of a leg receiving CMS10y semi-annually over the next 5 years 0.40056'

### Data prep for CMS 2y

In [6]:
tenor = 2
tenor2_forward = fsr.loc[fsr.Tenor == tenor]
tenor2_forward = tenor2_forward.copy()  # Make an explicit copy

new_expiry = np.arange(0, 10, 0.25)

df_2y_cms = pd.DataFrame({"Expiry": new_expiry, 
                           

                            "Tenor" : 2})

# interpolate sabr params
for col in ["alpha", "rho", "nu"]:
    interp_func = interp1d(tenor2_forward["Expiry"], 
                        tenor2_forward[col], 
                        kind="linear", 
                        fill_value="extrapolate")
    df_2y_cms[col] = interp_func(new_expiry)
# add par IRS  then interpolate par swap rates
tenor2_forward.loc[0] = {"Expiry": 0, "f_irs": 0.03}  # from excel sheet
tenor2_forward.sort_values(by="Expiry", inplace=True)
interp_func = interp1d(tenor2_forward["Expiry"], 
                    tenor2_forward['f_irs'], 
                    kind="linear", 
                    fill_value="extrapolate")
df_2y_cms['f_irs'] = interp_func(new_expiry)

df_2y_cms.head(2)

,Expiry,Tenor,alpha,rho,nu,f_irs
0,0.00,2,0.180946,0.519203,-1.828011,0.030000
1,0.25,2,0.181876,0.520592,-1.789689,0.030804


### Calculate PV of CMS 2y for 10 years

In [10]:
ls_v0 = []

for expiry in np.arange(0, 10, 0.25):
    if expiry == 0:
        coupon = df_2y_cms.iloc[0]['f_irs']
        ls_v0.append(coupon)       # first payment is fixed at par swap for first period
        print(f"{expiry}: {coupon}")
        continue

    F = df_2y_cms.loc[df_2y_cms.Expiry == expiry].f_irs.values[0]
    alpha = df_2y_cms.loc[df_2y_cms.Expiry == expiry].alpha.values[0]
    rho = df_2y_cms.loc[df_2y_cms.Expiry == expiry].rho.values[0]
    nu = df_2y_cms.loc[df_2y_cms.Expiry == expiry].nu.values[0]
    m = 4 # quarterly

    coupon = v_0(F, expiry, tenor, alpha, rho, nu, m)
    ls_v0.append(coupon)
    print(f"{expiry}: {coupon}")

f"PV of a leg receiving CMS10y semi-annually over the next 10 years {np.array(ls_v0).sum()}"

0.0: 0.03
0.25: 0.030808845289106736
0.5: 0.03163601052202935
0.75: 0.03247569509675975
1.0: 0.03332995173906539
1.25: 0.03381269922713553
1.5: 0.0342972866421102
1.75: 0.034780461781671594
2.0: 0.03526010582664303
2.25: 0.035734435101100345
2.5: 0.0362030604526493
2.75: 0.0366655260886644
3.0: 0.037121628800043625
3.25: 0.037567670360435616
3.5: 0.03800734971133321
3.75: 0.03844086297101213
4.0: 0.03886847425609437
4.25: 0.0392900179553345
4.5: 0.03970631724702558
4.75: 0.04011775377338323
5.0: 0.040524731775362616
5.25: 0.04067385488598534
5.5: 0.04082143150998483
5.75: 0.04096746116758574
6.0: 0.04111194754836154
6.25: 0.04125489767773775
6.5: 0.041396321254097077
6.75: 0.04153623011832558
7.0: 0.041674637826954575
7.25: 0.041806478348188784
7.5: 0.04193682636848922
7.75: 0.04206569887984528
8.0: 0.04219311354772403
8.25: 0.04231908855733261
8.5: 0.04244364248714222
8.75: 0.042566794204679285
9.0: 0.04268856278059488
9.25: 0.04280896741780934
9.5: 0.042928027393144115
9.75: 0.043045

'PV of a leg receiving CMS10y semi-annually over the next 10 years 1.5508886286002843'

In [11]:
# TODO make function to get CMS for given tenor